In [ ]:
using CSV
using DataFrames 
using Pipe
using OutlierDetection

In [ ]:
ls_loc = "../dataset/dataset/last_year/"
ls_tp = CSV.File(ls_loc*"LastYear_tp.csv") |> DataFrame
ls_ks = CSV.File(ls_loc*"LastYear_ks.csv") |> DataFrame
ls_nt = CSV.File(ls_loc*"LastYear_nt.csv") |> DataFrame
ls_tc = CSV.File(ls_loc*"LastYear_tc.csv") |> DataFrame
ls_tn = CSV.File(ls_loc*"LastYear_tn.csv") |> DataFrame
ls_ty = CSV.File(ls_loc*"LastYear_ty.csv") |> DataFrame
println("")